In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660605 sha256=0df3613e45d110d5961570afd2aa254f83e003024987f73d2f7382c4a4584502
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [2]:
# Импорт необходимых библиотек
import numpy as np  # Для работы с массивами
import pycuda.driver as cuda  # Для работы с CUDA
import pycuda.autoinit  # Автоматическая инициализация CUDA
from pycuda.compiler import SourceModule  # Для компиляции CUDA ядра
import struct  # Для работы с бинарными данными

In [3]:
# Конфигурация изображения
WIDTH = 1024  # Ширина изображения в пикселях
HEIGHT = 768  # Высота изображения в пикселях
MAX_DEPTH = 10  # Максимальная глубина рекурсии для трассировки лучей

In [6]:
# Функция для сохранения изображения в формате BMP
def save_bmp(filename, image, width, height):
    # Открываем файл для записи в бинарном режиме
    with open(filename, 'wb') as f:
        # Создаем заголовок BMP
        header = bytearray([
            66, 77,  # Сигнатура 'BM'
            0, 0, 0, 0,  # Размер файла (будет записан позже)
            0, 0,  # Зарезервировано
            0, 0,  # Зарезервировано
            54, 0, 0, 0,  # Смещение до массива пикселей
            40, 0, 0, 0,  # Размер DIB заголовка
            *struct.pack('<i', width),  # Ширина изображения
            *struct.pack('<i', height),  # Высота изображения
            1, 0,  # Количество цветовых плоскостей
            24, 0,  # Бит на пиксель (24 бита = 3 байта на пиксель)
            0, 0, 0, 0,  # Метод сжатия (0 = без сжатия)
            0, 0, 0, 0,  # Размер изображения (0 для несжатых изображений)
            0, 0, 0, 0,  # Горизонтальное разрешение (пикселей на метр)
            0, 0, 0, 0,  # Вертикальное разрешение (пикселей на метр)
            0, 0, 0, 0,  # Количество цветов в палитре (0 = все цвета)
            0, 0, 0, 0  # Количество важных цветов (0 = все важны)
        ])

        # Вычисляем размер файла и записываем его в заголовок
        file_size = len(header) + len(image)
        header[2:6] = struct.pack('<I', file_size)
        f.write(header)  # Записываем заголовок в файл

        # Записываем пиксели изображения
        for row in range(height - 1, -1, -1):  # BMP записывает строки снизу вверх
            for col in range(width):
                idx = (row * width + col) * 3  # Индекс пикселя в массиве
                b, g, r = image[idx], image[idx + 1], image[idx + 2]  # Получаем цвет (BGR)
                f.write(bytes([b, g, r]))  # Записываем цвет в файл

# Определение CUDA ядра
mod = SourceModule("""
    #include <math.h>  // Для математических функций
    #define MAX_DEPTH 10  // Максимальная глубина рекурсии
    #define PI 3.14159265358979323846  // Число Пи

    // Структура для представления 3D вектора
    struct Vec3 {
        float x, y, z;

        // Оператор сложения векторов
        __device__ Vec3 operator+(const Vec3& v) const {
            return {x + v.x, y + v.y, z + v.z};
        }

        // Оператор вычитания векторов
        __device__ Vec3 operator-(const Vec3& v) const {
            return {x - v.x, y - v.y, z - v.z};
        }

        // Умножение вектора на скаляр
        __device__ Vec3 operator*(float scalar) const {
            return {x * scalar, y * scalar, z * scalar};
        }

        // Поэлементное умножение векторов
        __device__ Vec3 operator*(const Vec3& v) const {
            return {x * v.x, y * v.y, z * v.z};
        }

        // Скалярное произведение векторов
        __device__ float dot(const Vec3& v) const {
            return x * v.x + y * v.y + z * v.z;
        }

        // Нормализация вектора
        __device__ Vec3 normalize() const {
            float len = sqrtf(dot(*this));  // Длина вектора
            return (len > 0) ? *this * (1.0f / len) : Vec3{0.0f, 0.0f, 0.0f};  // Нормализованный вектор
        }

        // Оператор += для сложения векторов
        __device__ Vec3& operator+=(const Vec3& v) {
            x += v.x;
            y += v.y;
            z += v.z;
            return *this;
        }
    };

    // Структура для представления сферы
    struct Sphere {
        Vec3 center;  // Центр сферы
        float radius;  // Радиус сферы
        Vec3 color;  // Цвет сферы
    };

    // Структура для представления источника света
    struct Light {
        Vec3 position;  // Позиция источника света
        Vec3 intensity;  // Интенсивность света
    };

    // Структура для представления плоскости
    struct Plane {
        Vec3 point;  // Точка на плоскости
        Vec3 normal;  // Нормаль к плоскости
        Vec3 color;  // Цвет плоскости
    };

    // Функция для проверки пересечения луча и сферы
    __device__ bool intersect(const Vec3& rayOrigin, const Vec3& rayDir, const Sphere& sphere, float& t) {
        Vec3 oc = rayOrigin - sphere.center;  // Вектор от центра сферы до начала луча
        float a = rayDir.dot(rayDir);  // Коэффициент a квадратного уравнения
        float b = 2.0f * oc.dot(rayDir);  // Коэффициент b квадратного уравнения
        float c = oc.dot(oc) - sphere.radius * sphere.radius;  // Коэффициент c квадратного уравнения
        float discriminant = b * b - 4 * a * c;  // Дискриминант

        if (discriminant < 0) return false;  // Нет пересечения, если дискриминант отрицательный
        t = (-b - sqrtf(discriminant)) / (2.0f * a);  // Находим ближайшее пересечение
        return t >= 0;  // Возвращаем true, если пересечение перед лучом
    }

    // Функция для проверки пересечения луча и плоскости
    __device__ bool intersectPlane(const Vec3& rayOrigin, const Vec3& rayDir, const Plane& plane, float& t) {
        float denom = plane.normal.dot(rayDir);  // Знаменатель для вычисления t
        if (fabs(denom) > 1e-6f) {  // Если луч не параллелен плоскости
            Vec3 p0l0 = plane.point - rayOrigin;  // Вектор от начала луча до точки на плоскости
            t = p0l0.dot(plane.normal) / denom;  // Вычисляем t
            return (t >= 0);  // Возвращаем true, если пересечение перед лучом
        }
        return false; // Нет пересечения
    }

    // Функция для трассировки луча
    __device__ Vec3 traceRay(const Vec3& rayOrigin, const Vec3& rayDir, Sphere* spheres, int numSpheres, Light* lights, int numLights, Plane* planes, int numPlanes, int depth) {
        if (depth > MAX_DEPTH) return {0.0f, 0.0f, 0.0f};  // Прерываем рекурсию, если достигнута максимальная глубина

        float closestT = 1e20f;  // Ближайшее пересечение
        int closestSphere = -1;  // Индекс ближайшей сферы
        int closestPlane = -1;  // Индекс ближайшей плоскости
        bool hitPlane = false;  // Флаг пересечения с плоскостью

        // Поиск ближайшего пересечения со сферами
        for (int i = 0; i < numSpheres; ++i) {
            float t;
            if (intersect(rayOrigin, rayDir, spheres[i], t) && t < closestT) {
                closestT = t;
                closestSphere = i;
            }
        }

        // Поиск ближайшего пересечения с плоскостями
        for (int i = 0; i < numPlanes; ++i) {
            float t;
            if (intersectPlane(rayOrigin, rayDir, planes[i], t) && t < closestT) {
                closestT = t;
                closestPlane = i;
                hitPlane = true;
            }
        }

        Vec3 color = {0.0f, 0.0f, 0.0f};  // Итоговый цвет

        // Обработка пересечения с плоскостью
        if (hitPlane && closestPlane != -1) {
            const Plane& plane = planes[closestPlane];  // Ближайшая плоскость
            Vec3 intersection = rayOrigin + rayDir * closestT;  // Точка пересечения
            Vec3 normal = plane.normal.normalize();  // Нормаль к плоскости

            color = plane.color * 0.1f;  // Базовый цвет (амбиентное освещение)

            // Добавляем освещение от источников света
            for (int i = 0; i < numLights; ++i) {
                Vec3 lightDir = (lights[i].position - intersection).normalize();  // Направление к свету
                float brightness = fmaxf(0.0f, normal.dot(lightDir));  // Яркость освещения
                color += plane.color * (lights[i].intensity * brightness);  // Добавляем цвет от света
            }
        }

        // Обработка пересечения со сферой
        if (closestSphere != -1) {
            const Sphere& sphere = spheres[closestSphere];  // Ближайшая сфера
            Vec3 intersection = rayOrigin + rayDir * closestT;  // Точка пересечения
            Vec3 normal = (intersection - sphere.center).normalize();  // Нормаль к сфере

            color = sphere.color * 0.1f;  // Базовый цвет (амбиентное освещение)

            // Добавляем освещение от источников света
            for (int i = 0; i < numLights; ++i) {
                Vec3 lightDir = (lights[i].position - intersection).normalize();  // Направление к свету
                bool inShadow = false;  // Флаг тени

                // Проверка на наличие тени
                for (int j = 0; j < numSpheres; ++j) {
                    float shadowT;
                    if (intersect(intersection + normal * 1e-4f, lightDir, spheres[j], shadowT)) {
                        inShadow = true;  // Объект в тени
                        break;
                    }
                }
                if (!inShadow) {
                    float brightness = fmaxf(0.0f, normal.dot(lightDir));  // Яркость освещения
                    color += sphere.color * (lights[i].intensity * brightness);  // Добавляем цвет от света
                }
            }
        }

        return color;  // Возвращаем итоговый цвет
    }

    // Ядро для рендеринга изображения
    __global__ void renderKernel(unsigned char* image, int width, int height, Sphere* spheres, int numSpheres, Light* lights, int numLights, Plane* planes, int numPlanes) {
        int x = blockIdx.x * blockDim.x + threadIdx.x;  // Координата x пикселя
        int y = blockIdx.y * blockDim.y + threadIdx.y;  // Координата y пикселя

        if (x >= width || y >= height) return;  // Выход, если координаты за пределами изображения

        Vec3 rayOrigin = {0.0f, 0.0f, 0.0f};  // Начало луча (камера)
        Vec3 rayDir = {(2.0f * (x + 0.5f) / width - 1.0f) * width / height, (1.0f - 2.0f * (y + 0.5f) / height), -1.0f};  // Направление луча
        rayDir = rayDir.normalize();  // Нормализация направления луча

        // Трассировка луча и получение цвета
        Vec3 color = traceRay(rayOrigin, rayDir, spheres, numSpheres, lights, numLights, planes, numPlanes, 0);

        // Запись цвета в массив изображения
        int pixelIndex = (y * width + x) * 3;  // Индекс пикселя
        image[pixelIndex] = (unsigned char)(fminf(1.0f, color.x) * 255);  // Красный канал
        image[pixelIndex + 1] = (unsigned char)(fminf(1.0f, color.y) * 255);  // Зеленый канал
        image[pixelIndex + 2] = (unsigned char)(fminf(1.0f, color.z) * 255);  // Синий канал
    }
""")

In [16]:
# Настройка данных для сцены
spheres = np.array([
    (np.array([0.0, -1.0, -5.0], dtype=np.float32), 1.5, np.array([1.0, 0.5, 0.0], dtype=np.float32)),  # Оранжевая сфера
    (np.array([-3.0, 0.0, -6.0], dtype=np.float32), 1.0, np.array([0.0, 1.0, 1.0], dtype=np.float32)),  # Голубая сфера
    (np.array([3.0, 0.0, -7.0], dtype=np.float32), 1.2, np.array([1.0, 0.0, 1.0], dtype=np.float32)),  # Фиолетовая сфера
    (np.array([0.0, 3.0, -8.0], dtype=np.float32), 1.0, np.array([1.0, 1.0, 0.0], dtype=np.float32)),  # Желтая сфера
    (np.array([-2.0, -2.0, -4.0], dtype=np.float32), 0.8, np.array([0.5, 0.5, 1.0], dtype=np.float32)),  # Светло-синяя сфера
], dtype=[('center', np.float32, 3), ('radius', np.float32), ('color', np.float32, 3)])

# Источники света
lights = np.array([
    (np.array([5.0, 5.0, -3.0], dtype=np.float32), np.array([1.0, 1.0, 1.0], dtype=np.float32)),  # Белый свет
    (np.array([0.0, 10.0, -5.0], dtype=np.float32), np.array([0.8, 0.8, 0.8], dtype=np.float32)),  # Верхний свет
], dtype=[('position', np.float32, 3), ('intensity', np.float32, 3)])

# Плоскости
planes = np.array([
    (np.array([0.0, -2.0, 0.0], dtype=np.float32), np.array([0.0, 1.0, 0.0], dtype=np.float32), np.array([0.8, 0.8, 0.8], dtype=np.float32)),  # Пол (серый)
], dtype=[('point', np.float32, 3), ('normal', np.float32, 3), ('color', np.float32, 3)])

In [17]:
# Выделение памяти на GPU и копирование данных
spheres_gpu = cuda.mem_alloc(spheres.nbytes)  # Выделяем память для сфер
cuda.memcpy_htod(spheres_gpu, spheres)  # Копируем данные на GPU

lights_gpu = cuda.mem_alloc(lights.nbytes)  # Выделяем память для источников света
cuda.memcpy_htod(lights_gpu, lights)  # Копируем данные на GPU

planes_gpu = cuda.mem_alloc(planes.nbytes)  # Выделяем память для плоскостей
cuda.memcpy_htod(planes_gpu, planes)  # Копируем данные на GPU

# Создание пустого изображения
image = np.zeros(WIDTH * HEIGHT * 3, dtype=np.uint8)  # Массив для хранения изображения (RGB)

# Настройка сетки и блоков для выполнения ядра
block_size = (16, 16, 1)  # Размер блока (16x16 потоков)
grid_size = (WIDTH // block_size[0], HEIGHT // block_size[1])  # Размер сетки блоков

# Запуск ядра рендеринга
render_kernel = mod.get_function("renderKernel")  # Получаем функцию ядра
render_kernel(
    cuda.Out(image),  # Выходной массив для изображения
    np.int32(WIDTH),  # Ширина изображения
    np.int32(HEIGHT),  # Высота изображения
    spheres_gpu,  # Сферы на GPU
    np.int32(spheres.shape[0]),  # Количество сфер
    lights_gpu,  # Источники света на GPU
    np.int32(lights.shape[0]),  # Количество источников света
    planes_gpu,  # Плоскости на GPU
    np.int32(planes.shape[0]),  # Количество плоскостей
    block=block_size,  # Размер блока
    grid=grid_size  # Размер сетки
)

# Сохранение изображения в файл
save_bmp('output.bmp', image, WIDTH, HEIGHT)  # Сохраняем изображение в формате BMP

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
